Cuando haya que re-entrenar, esta será la notebook que tendremos que utilizar

Objetivo: que se pueda correr por consola

In [37]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from pandas.io import gbq
from datetime import datetime
import warnings
import openpyxl
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
#!pip install sklearn-pandas
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import r2_score

## 1) Construccion del pipelinetarget

In [2]:
# Transformer de capeo
def outlier_cap(data):
    return pd.DataFrame(data).apply(lambda x: x.clip(upper=x.quantile(0.99))) 

outlier_transformer = FunctionTransformer(outlier_cap)

In [3]:
# Identificamos features
cols_num = ['car_kms','car_year'] # estas hay que capearlas al p99 solo en el pipe de scoreo, porque en el ds de desarrollo directamente dropeamos los registros
cols_categ = ['match_marca_a','match_modelo_a','match_v1_c', 'Subseg_a', 'Seg_a'] # estas hay que sumarle el sufijo de las categoricas

In [4]:
# Sufijo necesario para usar catboost adentro de un pipeline
categorical_suffix = "_#CAT#"

In [5]:
# Creamos el column transformer, el cual será el step 1 del pipeline
#ct1 = make_column_transformer(
#    (outlier_transformer, cols_num)
#    ,remainder='passthrough')

In [6]:
# Reemplaza a un "make_column_transformer" y el diferencial es que el output lo tira en df y no en ndarray, lo cual nos complicaba aplicar el
# catboost en el 2 step del pipeline, ya que debemos poder identificar las categoricas y para eso necesitabamos los nombres de las cols
mapper = DataFrameMapper([
    (cols_num, outlier_transformer),
    #(['seasons'], SimpleImputer(strategy='median')),
    #('genre', [CategoricalImputer(strategy='constant', fill_value='unknown'),
    #           LabelBinarizer()]),
    #('on_netflix', [CategoricalImputer(strategy='constant', fill_value='unknown'),
    #               LabelEncoder()])
], df_out=True)

**Custom CatBoost Classifier** <br>
We need to implement our own catboost classifier so we can track our categorical features

In [7]:
class CustomCatBoostRegressor(CatBoostRegressor):

    def fit(self, X, y=None, **fit_params):
        print(X.filter(regex=f"{categorical_suffix}$").columns.to_list())

        return super().fit(
            X,
            y=y,
            cat_features=X.filter(regex=f"{categorical_suffix}$").columns,
            **fit_params
        )

In [8]:
# Instanciamos el modelo, el cual será el step 2 del pipline
#### aca no sería mejor usar el pickle con el modelo ya entrenado???? ######
CB_estimator = CustomCatBoostRegressor(iterations=50, depth=3, learning_rate=0.1, loss_function='RMSE') #,logging_level="Silent"

**Pipeline final**

In [9]:
pipe = make_pipeline(mapper, CB_estimator)

In [62]:
pipe.get_params()

{'memory': None,
 'steps': [('dataframemapper',
   DataFrameMapper(df_out=True, drop_cols=[],
                   features=[(['car_kms', 'car_year'],
                              FunctionTransformer(func=<function outlier_cap at 0x0000014FDF351940>))])),
  ('customcatboostregressor',
   <__main__.CustomCatBoostRegressor at 0x14fdf3530a0>)],
 'verbose': False,
 'dataframemapper': DataFrameMapper(df_out=True, drop_cols=[],
                 features=[(['car_kms', 'car_year'],
                            FunctionTransformer(func=<function outlier_cap at 0x0000014FDF351940>))]),
 'customcatboostregressor': <__main__.CustomCatBoostRegressor at 0x14fdf3530a0>,
 'dataframemapper__default': False,
 'dataframemapper__df_out': True,
 'dataframemapper__drop_cols': [],
 'dataframemapper__features': [(['car_kms', 'car_year'],
   FunctionTransformer(func=<function outlier_cap at 0x0000014FDF351940>))],
 'dataframemapper__input_df': False,
 'dataframemapper__sparse': False,
 'customcatboostregressor__

## 2) Dataset de desarrollo

In [10]:
# Upload del pickle
df_meli = pd.read_pickle('df_meli_2021-12-20.pkl')

In [11]:
df_meli.shape

(3104549, 42)

**Todo lo que viene ahora es lo que tenemos que tratar de que este directamente en la query de sql que importa la data de BigQuery**

**Removing duplicates**

In [12]:
# Tratamiento de la feautre "runtime"
df_meli['runtime'] = pd.to_datetime(df_meli.runtime.apply(lambda x: str(x)[:19]))
df_meli = df_meli.sort_values(['runtime'])
# Ahora si, teniendo el df ordenado, eliminamos los duplicados, quedandonos con el registro más reciente (la última ocurrencia)
df_meli.drop_duplicates(subset=['car_id'], keep='last', inplace=True)

**Precio final**

In [13]:
# Construcción del precio final
blue= 196.5
oficial= 108.4
col1 = 'price_symbol'
col2 = 'car_kms'
conditions = [df_meli[col1]!='U$S', (df_meli[col1]=='U$S') & (df_meli[col2]==0), (df_meli[col1]=='U$S') & (df_meli[col2]!=0)]
choices = [df_meli.price_amount, df_meli['price_amount']*oficial, df_meli['price_amount']*blue]

df_meli['price_meli_ok'] = np.select(conditions, choices, default=np.nan)

In [14]:
# Luego de ya tener el precio final calculado, podemos eliminar del dataset las variables de precio originales
df_meli.drop(['price_symbol','price_amount'],1,inplace=True)

**Remove outliers**

In [15]:
# Analisis: Excluir outliers de "price_meli_ok" --> directamente dropeamos los registros
q_99_p = np.quantile(df_meli['price_meli_ok'],.99)
q_99_km = np.quantile(df_meli['car_kms'],.99)
df_meli = df_meli[(df_meli.price_meli_ok <= q_99_p) & (df_meli.car_kms <= q_99_km)]

**Removing catgories**

In [16]:
cats_loc1_ok = ['Capital Federal','Bs.As. G.B.A. Sur','Bs.As. G.B.A. Norte'
           ,'Bs.As. G.B.A. Oeste','Buenos Aires Interior','Bs.As. Costa Atlántica']
mask = df_meli.car_location_1.apply(lambda x: x in cats_loc1_ok)
df_meli = df_meli[mask]

In [17]:
cats_brand_ok =['Alfa Romeo','Audi','BMW','Chery','Chevrolet','Chrysler','Citroën','DS','Dodge','Fiat','Ford','Geely','Haval','Honda',
 'Hyundai','JAC','Jeep','Kia','Land Rover','Lifan','Mercedes-Benz','Mini','Mitsubishi','Nissan','Peugeot','Porsche','RAM',
 'Renault','SEAT','Shineray','Smart','Subaru','Suzuki','Toyota','Volkswagen','Volvo']
mask = df_meli.brand.apply(lambda x: x in cats_brand_ok)
df_meli = df_meli[mask]

**Removing 0km & concesionaria**

In [18]:
# Pasamos "dealer" de booleano a flag 1 y 0
df_meli['dealer'] = np.where(df_meli['dealer']==True,1,0)
mask_not_0km = df_meli.car_kms > 90
mask_not_conces = df_meli.dealer == 0

df_meli = df_meli[(mask_not_0km) & (mask_not_conces)]

# Como nos quedamos con solamente un valor de la feature "dealer" la podemos eliminar
df_meli.drop('dealer',1,inplace=True)

**Removing match_scores under 50%**

In [19]:
lst = ['score_marca_a','score_modelo_a','score_v1_c']
for col in lst:
    df_meli = df_meli[df_meli[col]>=50]
    
# chequeo
df_meli[lst].min()

score_marca_a     92.0
score_modelo_a    50.0
score_v1_c        50.0
dtype: float64

**Removing nulls**

In [22]:
# Eliminamos los registros con nulos
for col in ['car_year','car_kms','match_marca_a','match_modelo_a','match_v1_c', 'Subseg_a', 'Seg_a']:
    df_meli = df_meli[~df_meli[col].isnull()]

**Le agregamos el sufijo a las categoricas** <br>
Esto es para que luego el custom catboost reconozca cuales son las categoricas

In [23]:
for col in cols_categ:
    df_meli.rename(columns={col:f'{col}{categorical_suffix}'}, inplace=True)

**Selecting final feautres**

In [25]:
final_features = ['car_year','car_kms','match_marca_a_#CAT#','match_modelo_a_#CAT#'
                  ,'match_v1_c_#CAT#', 'Subseg_a_#CAT#', 'Seg_a_#CAT#']
car_location = ['car_location_1'] # esta no la usamos para entrenar el modelo, solo que la queremos en el output final
car_ids = ['car_id'] # lo necesitamos para que las simulaciones no scoreen registros que ya se scorearon antes
target = ['price_meli_ok']
df_meli_final = df_meli[final_features+car_location+car_ids+target]

In [26]:
# Ajustamos un dtype
# esto simplemente lo hacemos para despues en el catboost diferenciar entre float y object, en lugar de sumar int. 
df_meli_final['car_year']=df_meli_final['car_year'].astype('float')

In [27]:
# chequeo post limpieza
df_meli_final.shape

(37511, 10)

In [28]:
# chequeo post limpieza
df_meli_final.head()

,car_year,car_kms,match_marca_a_#CAT#,match_modelo_a_#CAT#,match_v1_c_#CAT#,Subseg_a_#CAT#,Seg_a_#CAT#,car_location_1,car_id,price_meli_ok
1640282,2019.0,32000.0,CHEVROLET,Onix,JOY 5P 1.4 N LS MT,Gama Pequenos Hatchback,Gama Pequenos,Bs.As. G.B.A. Sur,1104596590,1500000.0
2292449,2020.0,9000.0,FORD,Ranger,LIMITED 4X4 2.8D CAB.DOB.,Pickups Medianas-Grandes,Livianos,Buenos Aires Interior,1108782314,6300000.0
1361521,2020.0,10000.0,FORD,Ranger,LIMITED 4X4 2.8D CAB.DOB.,Pickups Medianas-Grandes,Livianos,Bs.As. Costa Atlántica,1109381735,6900000.0
581742,2020.0,50000.0,FORD,KA,S 1.5L,Gama Pequenos Hatchback,Gama Pequenos,Bs.As. G.B.A. Sur,1105093008,1600000.0
597897,2007.0,120000.0,PEUGEOT,206,XR PRESENCE 1.4 3P,Gama Pequenos Hatchback,Gama Pequenos,Bs.As. G.B.A. Oeste,1104791935,600000.0


*- Fin: Dataset de desarrollo -*

## 3) Split train test

In [65]:
X = df_meli_final[final_features]
y = df_meli_final.price_meli_ok
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

## 4) Fit pipeline

In [66]:
# Fiteamos el pipeline
pipe.fit(X_train, y_train)

[]
0:	learn: 1656330.8108532	total: 3.39ms	remaining: 166ms
1:	learn: 1616480.2324321	total: 6.69ms	remaining: 161ms
2:	learn: 1581375.5202441	total: 10.1ms	remaining: 158ms
3:	learn: 1553205.4304882	total: 13.9ms	remaining: 160ms
4:	learn: 1528307.0230616	total: 17.8ms	remaining: 160ms
5:	learn: 1507487.8496492	total: 21.3ms	remaining: 156ms
6:	learn: 1490349.4967818	total: 24.7ms	remaining: 151ms
7:	learn: 1475637.6061757	total: 27.8ms	remaining: 146ms
8:	learn: 1463140.6826816	total: 31.1ms	remaining: 142ms
9:	learn: 1453173.6869814	total: 34.6ms	remaining: 138ms
10:	learn: 1444170.2466628	total: 37.8ms	remaining: 134ms
11:	learn: 1436673.7685763	total: 40.9ms	remaining: 130ms
12:	learn: 1430645.4593103	total: 44.1ms	remaining: 126ms
13:	learn: 1425241.2298308	total: 47.3ms	remaining: 122ms
14:	learn: 1420746.8963229	total: 50.7ms	remaining: 118ms
15:	learn: 1416519.9556672	total: 53.6ms	remaining: 114ms
16:	learn: 1412834.1115794	total: 56.7ms	remaining: 110ms
17:	learn: 1409951.99

Pipeline(steps=[('dataframemapper',
                 DataFrameMapper(df_out=True, drop_cols=[],
                                 features=[(['car_kms', 'car_year'],
                                            FunctionTransformer(func=<function outlier_cap at 0x0000014FDF351940>))])),
                ('customcatboostregressor',
                 <__main__.CustomCatBoostRegressor object at 0x0000014FDF3530A0>)])

In [56]:
print("\nTest  R2 : %.3f"%pipe.score(X_test, y_test))
print("Train R2 : %.3f"%pipe.score(X_train, y_train))


Test  R2 : -9992.164
Train R2 : -39623.601


In [73]:
# R2

# Train
#print(f'Train: {r2_score(y_true_train, y_pred_train)}')

# Test
y_true_test = y_train
y_pred_test = pipe.predict(X_train)
print(f'Test: {r2_score(y_true_test, y_pred_test)}')

Test: 0.3328259877340365


In [71]:
categorical_features_indices = np.where(X_train.dtypes != np.float)[0]

In [72]:
########### prueba sin el eval_test parameter ##########
model=CatBoostRegressor(iterations=50, depth=3, learning_rate=0.1, loss_function='RMSE') #,logging_level="Silent"
model.fit(X_train, y_train, cat_features=categorical_features_indices)

test_preds = model.predict(X_test)
train_preds = model.predict(X_train)

from catboost.utils import eval_metric

print("\nTest  R2 : %.3f"%model.score(X_test, y_test))
print("Train R2 : %.3f"%model.score(X_train, y_train))

0:	learn: 1603588.0303176	total: 13.2ms	remaining: 645ms
1:	learn: 1516259.7668622	total: 24.6ms	remaining: 591ms
2:	learn: 1440527.9550434	total: 35.5ms	remaining: 556ms
3:	learn: 1370836.3687444	total: 45.5ms	remaining: 523ms
4:	learn: 1310022.2865467	total: 57.5ms	remaining: 518ms
5:	learn: 1258334.0189364	total: 67.4ms	remaining: 494ms
6:	learn: 1209239.0034789	total: 77.8ms	remaining: 478ms
7:	learn: 1162992.3326794	total: 87.9ms	remaining: 462ms
8:	learn: 1126269.0580224	total: 97.9ms	remaining: 446ms
9:	learn: 1093539.6082587	total: 108ms	remaining: 433ms
10:	learn: 1063255.1134896	total: 119ms	remaining: 421ms
11:	learn: 1036228.5248364	total: 128ms	remaining: 407ms
12:	learn: 1009672.1892504	total: 138ms	remaining: 393ms
13:	learn: 988361.1772137	total: 149ms	remaining: 382ms
14:	learn: 969549.0709939	total: 159ms	remaining: 370ms
15:	learn: 951482.0177084	total: 172ms	remaining: 365ms
16:	learn: 935086.2642648	total: 181ms	remaining: 352ms
17:	learn: 920498.6297818	total: 192

In [68]:
df_meli_final.drop('price_meli_ok',1).columns

Index(['car_year', 'car_kms', 'match_marca_a_#CAT#', 'match_modelo_a_#CAT#',
       'match_v1_c_#CAT#', 'Subseg_a_#CAT#', 'Seg_a_#CAT#', 'car_location_1',
       'car_id'],
      dtype='object')